In [ ]:
import numpy as np
from scipy.io import mmread
# from scipy.sparse import csr_matrix # Non usata se A è densa
import time
from scipy.optimize import minimize_scalar

def projection_simplex(v, z=1.0):
    """Project vector v onto the simplex (sum(x)=z, x>=0)."""
    n = len(v)
    if n == 0: return np.array([])
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    
    # Trova rho: l'indice più grande tale che u[rho-1] - (cssv[rho-1] - z) / rho > 0
    # (Usando indicizzazione 0-based per rho_idx, quindi rho = rho_idx + 1)
    rho_idx = -1
    for i in range(n - 1, -1, -1): # Itera da n-1 giù fino a 0
        if u[i] + (z - cssv[i]) / (i + 1) > 0: # cssv[i] è sum_{j=0 to i} u[j]
            rho_idx = i
            break
    
    # Se nessun rho trovato (es. tutti gli u_i sono molto negativi e z piccolo),
    # rho_idx rimane -1. Questo non dovrebbe accadere per z > 0 e input ragionevoli.
    # In Duchi et al. 2008, si dimostra che rho >= 1.
    if rho_idx == -1: # Fallback nel caso improbabile (es. z=0, tutti v<=0)
        if z == 0: return np.zeros(n)
        # Se z > 0, la teoria dice che rho >= 1 (cioè rho_idx >= 0)
        # Questo fallback è per estrema robustezza o casi degeneri.
        # Se tutti v_i sono <=0 e z>0, un vertice dovrebbe prendere tutto il peso.
        # Per semplicità, se non troviamo rho, e z > 0, è una situazione anomala.
        # L'implementazione originale con `ind[cond][-1]` era più concisa.
        # Per ora, assumiamo che rho_idx venga trovato se z > 0.
        # Se z>0 e non si trova rho, significa che tutti u_i + (z - cssv_i)/(i+1) <=0
        # Considera un caso semplice: v = [-1, -2], z = 1. u = [-1, -2]. cssv = [-1, -3].
        # i=1 (rho_idx=1): u[1] + (1 - cssv[1]) / 2 = -2 + (1 - (-3)) / 2 = -2 + 4/2 = 0. Non > 0.
        # i=0 (rho_idx=0): u[0] + (1 - cssv[0]) / 1 = -1 + (1 - (-1)) / 1 = -1 + 2 = 1. > 0. rho_idx = 0.
        # Quindi rho = 1.
        # Se z > 0, rho_idx sarà almeno 0.
        pass # rho_idx dovrebbe essere >=0

    # theta = (sum_{i=0 to rho_idx} u[i] - z) / (rho_idx + 1)
    theta = (cssv[rho_idx] - z) / (rho_idx + 1.0)
    
    proj_v = np.maximum(v - theta, 0)
    
    # Opzionale: normalizzazione finale per assicurare sum(x)=z a causa di errori numerici
    current_sum = np.sum(proj_v)
    if z > 1e-9: # Evita divisione per zero se z è ~0
        if abs(current_sum - z) > 1e-7 * z: # Controlla differenza relativa
            if current_sum > 1e-9: # Evita divisione per zero se current_sum è ~0
                proj_v = proj_v * (z / current_sum)
            else: # Se la somma è zero ma dovrebbe essere positiva, distribuisci z
                  # Questo indica un problema o un caso degenere.
                  # Ad es. assegna a un elemento o uniformemente (violerebbe sparsità)
                  # Per ora, ci fidiamo dell'algoritmo sopra, ma questo è un fallback.
                  # print(f"Avviso: somma proiezione simplesso {current_sum} diversa da target {z}.")
                  pass
    elif abs(current_sum) > 1e-7 : # se z=0, current_sum dovrebbe essere 0
         proj_v = np.zeros(n)
         
    return proj_v


def lmo(grad):
    """Linear Minimization Oracle for the simplex."""
    i = np.argmin(grad)
    s = np.zeros_like(grad)
    s[i] = 1.0
    return s, i # Restituisce anche l'indice per comodità

def f_l2(x, A):
    """L2-regularized objective function (da MASSIMIZZARE)."""
    return x @ A @ x + 0.5 * np.dot(x, x)

def grad_l2(x, A):
    """Gradient for L2-regularized objective (per MASSIMIZZAZIONE)."""
    # Il gradiente di x^T A x è (A+A^T)x. Se A è simmetrica, 2Ax.
    # Assumiamo A sia la matrice di adiacenza, quindi simmetrica.
    return 2 * (A @ x) + x

def f_l0(x, A, alpha=0.07, beta=5):
    """L0-regularized objective function (da MASSIMIZZARE)."""
    # Il -len(x) nel tuo codice originale per exp_term è insolito.
    # clustering.pdf Eq 33 è: α_2 Σ(e^(-βx_i) - 1)
    # Quindi dovrebbe essere: alpha * np.sum(np.exp(-beta * x) - 1.0)
    reg_term = alpha * np.sum(np.exp(-beta * np.clip(x, 0, None)) - 1.0) # Clip x>=0 per stabilità exp
    return x @ A @ x + reg_term

def grad_l0(x, A, alpha=0.07, beta=5):
    """Gradient for L0-regularized objective (per MASSIMIZZAZIONE)."""
    # Gradiente di α_2 Σ(e^(-βx_i) - 1) è -alpha * beta * exp(-beta * x_i) per ogni componente
    return 2 * (A @ x) - alpha * beta * np.exp(-beta * np.clip(x, 0, None))


def line_search_generic(objective_func, x_curr, d_curr, A_matrix, gamma_max=1.0, reg_type_info='l2', alpha_l0=0.07, beta_l0=5):
    """Line search generica usando minimize_scalar. Minimizza -objective_func."""
    if np.linalg.norm(d_curr) < 1e-12: return 0.0
    
    # objective_func è la funzione da MASSIMIZZARE
    # minimize_scalar minimizza, quindi gli passiamo -objective_func
    def func_to_minimize(gamma_ls):
        x_candidate = x_curr + gamma_ls * d_curr
        # Assicurati che x_candidate rimanga non negativo se necessario per f_l0
        if reg_type_info == 'l0':
            x_candidate = np.maximum(x_candidate, 0) # L0 reg ha exp(-beta*x)
        
        val = objective_func(x_candidate, A_matrix) if reg_type_info == 'l2' else \
              objective_func(x_candidate, A_matrix, alpha=alpha_l0, beta=beta_l0)
        return -val # Minimizziamo il negativo
    
    # Assicura che gamma_max_val sia un valore valido e non negativo.
    if not (np.isfinite(gamma_max) and gamma_max >= 0): gamma_max = 1.0
    if gamma_max < 1e-12: return 0.0

    try:
        res = minimize_scalar(func_to_minimize, bounds=(0, gamma_max), method='bounded')
        gamma = res.x
        if not np.isfinite(gamma): gamma = 0.0
    except Exception:
        gamma = 0.0 # Fallback
        
    return np.clip(gamma, 0.0, gamma_max)


def frank_wolfe(A, reg_type='l2', max_iter=1000, tol=1e-6):
    n = A.shape[0]
    x = np.ones(n) / n # Inizia dal centroide del simplesso
    
    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0
    
    for t in range(max_iter):
        grad_orig = gradient_function(x, A) # Gradiente dell'obiettivo di MASSIMIZZAZIONE
        grad_min = -grad_orig              # Gradiente dell'obiettivo di MINIMIZZAZIONE (-f_orig)
        
        s_vertex, _ = lmo(grad_min) # s = argmin <s, grad_min>
        d = s_vertex - x
        
        # FW Gap per il problema di MASSIMIZZAZIONE: <grad_orig, s - x>
        gap = grad_orig @ d 
        
        if gap < tol:
            # print(f"FW iter {t}: Gap {gap:.2e} < tol. Converged.")
            break
            
        if reg_type == 'l2':
            gamma = line_search_generic(objective_function, x, d, A, gamma_max=1.0, reg_type_info='l2')
        else: # Per L0, usa lo step size standard di FW o line search
            # gamma = 2.0 / (t + 2.0)
             gamma = line_search_generic(objective_function, x, d, A, gamma_max=1.0, reg_type_info='l0')

        x = x + gamma * d
    return x

def pairwise_frank_wolfe(A, reg_type='l2', max_iter=1000, tol=1e-6):
    n = A.shape[0]
    x0_idx = 0 
    x = np.zeros(n)
    x[x0_idx] = 1.0 # Inizia da un vertice
    
    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0
    
    # L'active_set tiene traccia degli indici degli atomi (vertici e_i)
    # che hanno peso non nullo in x. x stesso contiene i pesi.
    # active_set_indices = {x0_idx} # Non strettamente necessario se x è denso

    for t in range(max_iter):
        grad_orig = gradient_function(x, A)
        grad_min = -grad_orig
        
        s_vertex_fw, s_index = lmo(grad_min) # Atomo FW s_t
        
        # FW Gap per criterio di arresto (basato su massimizzazione)
        d_for_gap = s_vertex_fw - x
        gap = grad_orig @ d_for_gap
        if gap < tol:
            # print(f"PFW iter {t}: FW Gap {gap:.2e} < tol. Converged.")
            break
            
        # Atomo Away v_t: argmax_{v in current support of x} <grad_orig, v>
        # Cioè, trova l'indice i nel supporto di x per cui grad_orig[i] è massimo.
        current_support_indices = np.where(x > 1e-9)[0]
        if len(current_support_indices) == 0:
            # print(f"PFW iter {t}: Supporto di x vuoto! Reset anomalo a s_index.")
            x.fill(0.0); x[s_index] = 1.0
            # active_set_indices = {s_index}
            continue

        v_index = current_support_indices[np.argmax(grad_orig[current_support_indices])]
        
        if s_index == v_index:
            # Passo PFW nullo, potremmo fare un passo FW standard o saltare
            # print(f"PFW iter {t}: s_index == v_index. Using FW-like step.")
            d = s_vertex_fw - x # Direzione FW
            gamma_max_ls = 1.0
        else:
            v_vertex = np.zeros(n); v_vertex[v_index] = 1.0
            d = s_vertex_fw - v_vertex # Direzione Pairwise
            gamma_max_ls = x[v_index] # Peso alpha_v_t

        gamma = line_search_generic(objective_function, x, d, A, gamma_max=gamma_max_ls, reg_type_info=reg_type)
            
        # Aggiorna x direttamente (poiché x contiene i pesi alpha_i per gli atomi e_i)
        if s_index == v_index: # Logica di aggiornamento tipo FW
             x = (1.0 - gamma) * x + gamma * s_vertex_fw
        else: # Logica di aggiornamento Pairwise
            x[s_index] += gamma
            x[v_index] -= gamma
        
        # Pulisci pesi molto piccoli e normalizza (opzionale ma buono per stabilità)
        x[x < 1e-10] = 0.0
        current_sum = np.sum(x)
        if abs(current_sum - 1.0) > 1e-7 and current_sum > 1e-9:
            x /= current_sum
            
    return x

def away_step_frank_wolfe(A, reg_type='l2', max_iter=1000, tol=1e-6):
    n = A.shape[0]
    x0_idx = 0
    x = np.zeros(n)
    x[x0_idx] = 1.0

    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0

    for t in range(max_iter):
        grad_orig = gradient_function(x, A)
        grad_min = -grad_orig

        s_vertex_fw, s_index = lmo(grad_min) # Atomo FW s_t
        d_fw = s_vertex_fw - x
        
        # FW Gap per criterio di arresto (basato su massimizzazione)
        gap_fw = grad_orig @ d_fw
        if gap_fw < tol:
            # print(f"AFW iter {t}: FW Gap {gap_fw:.2e} < tol. Converged.")
            break

        # Atomo Away v_t: argmax_{v in current support of x} <grad_orig, v>
        current_support_indices = np.where(x > 1e-9)[0]
        if len(current_support_indices) == 0:
            # print(f"AFW iter {t}: Supporto di x vuoto! Reset anomalo a s_index.")
            x.fill(0.0); x[s_index] = 1.0
            continue
            
        v_index = current_support_indices[np.argmax(grad_orig[current_support_indices])]
        v_vertex = np.zeros(n); v_vertex[v_index] = 1.0
        d_away = x - v_vertex
        
        # Progresso potenziale per la MASSIMIZZAZIONE: <grad_orig, d>
        # Scegli la direzione che massimizza <grad_orig, d>
        potential_progress_fw = grad_orig @ d_fw
        potential_progress_away = grad_orig @ d_away
        
        direction_type = ""
        if potential_progress_fw >= potential_progress_away:
            d = d_fw
            gamma_max_ls = 1.0
            direction_type = "FW"
        else:
            d = d_away
            alpha_v_t = x[v_index]
            if abs(1.0 - alpha_v_t) < 1e-9 or np.linalg.norm(d_away) < 1e-9:
                # Se d_away è zero (x è v_vertex) o alpha_v_t è 1, gamma_max per away non è ben def.
                # In questo caso, un passo FW è più sensato se d_fw non è anch'esso zero.
                d = d_fw
                gamma_max_ls = 1.0
                direction_type = "FW (fallback from Away)"
            else:
                gamma_max_ls = alpha_v_t / (1.0 - alpha_v_t)
                direction_type = "Away"
        
        gamma = line_search_generic(objective_function, x, d, A, gamma_max=gamma_max_ls, reg_type_info=reg_type)

        # Aggiornamento di x (alpha_i)
        if direction_type == "FW" or direction_type == "FW (fallback from Away)":
            # x_new = (1-gamma)*x + gamma*s_vertex_fw
            x = (1.0 - gamma) * x
            x[s_index] += gamma
        elif direction_type == "Away":
            # x_new = (1+gamma)*x - gamma*v_vertex
            x = (1.0 + gamma) * x
            x[v_index] -= gamma
        
        x[x < 1e-10] = 0.0
        current_sum = np.sum(x)
        if abs(current_sum - 1.0) > 1e-7 and current_sum > 1e-9:
            x /= current_sum
            
    return x

def projected_gradient(A, reg_type='l2', max_iter=1000, tol=1e-6, lr_initial=0.1):
    n = A.shape[0]
    x = np.ones(n) / n
    
    objective_function = f_l2 if reg_type == 'l2' else f_l0
    gradient_function = grad_l2 if reg_type == 'l2' else grad_l0
    
    # Semplice backtracking line search per PG (adattato per massimizzazione)
    lr = lr_initial
    beta_bt = 0.5  # Fattore di riduzione per lr
    c_bt = 1e-4    # Costante Armijo

    for t in range(max_iter):
        grad_orig = gradient_function(x, A) # Gradiente per massimizzazione
        
        # Backtracking line search
        lr_current_iter = lr
        f_x_current = objective_function(x, A) if reg_type=='l2' else objective_function(x,A,alpha=0.07,beta=5) # Valore corrente per massimizzazione

        while True:
            x_candidate_step = x + lr_current_iter * grad_orig # Passo di salita del gradiente
            x_next = projection_simplex(x_candidate_step)      # Proiezione
            
            f_x_next = objective_function(x_next, A) if reg_type=='l2' else objective_function(x_next,A,alpha=0.07,beta=5)

            # Condizione di Armijo per massimizzazione: f(x_new) >= f(x_old) + c * <grad_f_old, x_new - x_old>
            if f_x_next >= f_x_current + c_bt * np.dot(grad_orig, x_next - x):
                break
            lr_current_iter *= beta_bt
            if lr_current_iter < 1e-9: # Evita step troppo piccoli / loop infiniti
                # print(f"PG iter {t}: Line search lr too small. Breaking LS.")
                break
        
        if np.linalg.norm(x_next - x) < tol:
            # print(f"PG iter {t}: Change in x {np.linalg.norm(x_next - x):.2e} < tol. Converged.")
            break
        x = x_next
        # Adatta lr per la prossima iterazione (euristica comune, opzionale)
        # if lr_current_iter < lr / (beta_bt * 2) and lr_current_iter > 1e-8 : lr = lr_current_iter * 2 
        # else: lr = lr_initial * 0.99 # Decadimento lento


    return x

# ... (il resto del codice per extract_clique, load_graph, e il blocco if __name__ == "__main__"
#      può rimanere come nella tua versione precedente, assicurandoti che chiami
#      le funzioni corrette e gestisca i loro output).



def extract_clique(x, A, threshold=1e-5):
    """Extract clique from solution vector."""
    S = np.where(x > threshold)[0]
    if len(S) == 0:
        return []
    
    # Sort by weight descending
    sorted_indices = S[np.argsort(-x[S])]
    clique = []
    for i in sorted_indices:
        # Check if vertex i is connected to all vertices in current clique
        is_connected = True
        for j in clique:
            if A[i, j] == 0:
                is_connected = False
                break
        if is_connected:
            clique.append(i)
    return clique

# Load graph from MTX file
def load_graph(file_path):
    """Load graph from MTX file and return adjacency matrix."""
    sparse_matrix = mmread(file_path)
    A_dense = sparse_matrix.toarray()
    np.fill_diagonal(A_dense, 0)  # Remove self-loops
    return A_dense

# Main experiment
if __name__ == "__main__":
    # Configuration
    graph_file = "C:/Users/ricky/OneDrive/Desktop/OPT_PROJECT/data/brock200-4.mtx"
    reg_types = ['l2', 'l0']
    algorithms = {
        'FW': frank_wolfe,
        'PFW': pairwise_frank_wolfe,
        'AFW': away_step_frank_wolfe,
        'PGD': projected_gradient
    }
    
    # Load graph
    A = load_graph(graph_file)
    n = A.shape[0]
    
    # Run experiments
    results = []
    for reg_type in reg_types:
        for algo_name, algo_func in algorithms.items():
            try:
                start_time = time.perf_counter()  # More precise timing
                x = algo_func(A, reg_type=reg_type)
                runtime = time.perf_counter() - start_time
                clique = extract_clique(x, A)
                clique_size = len(clique)
                
                # Verify clique validity
                is_valid = all(A[i, j] == 1 for i in clique for j in clique if i != j)
                
                results.append({
                    'Algorithm': algo_name,
                    'Regularization': reg_type,
                    'Clique Size': clique_size,
                    'Valid Clique': is_valid,
                    'Runtime (s)': runtime
                })
            except Exception as e:
                print(f"Error in {algo_name} with {reg_type}: {str(e)}")
    
    # Print results
    print("\nResults for", graph_file)
    print("="*60)
    print(f"Graph size: {n}x{n}")
    print("="*60)
    for res in results:
        print(f"{res['Algorithm']} + {res['Regularization']}:")
        print(f"  Clique Size = {res['Clique Size']}")
        print(f"  Valid Clique = {res['Valid Clique']}")
        print(f"  Runtime = {res['Runtime (s)']:.6f} s\n")